In [9]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
agg_func = ['std', 'skew', 'kurtosis', 'mean', 'median', 'var', 'mad', 'prod', 'sum']

In [3]:
def isMultiMode(df, col):
    m = df[col].mode()
    if m.size == 1:
        return False
    else:
        return True

In [4]:
def getMultiModes(df, cols):
    index = cols
    res = []
    for col in cols:
        res.append(isMultiMode(df, col))
    return res

In [21]:
def dropOutlierIQR(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col][df[col].between((Q1 - 1.5 * IQR), (Q3 + 1.5 * IQR))]

In [16]:
def dropOutliersIQR(df, cols):
    for col in cols:
        dropOutlierIQR(df, col)

In [6]:
# TODO use stats.zscore instead, ex: z = np.abs(stats.zscore(boston_df))
#  then np.where(z > 3)
def dropOutlierZscore(df, col):
    sigma3P = df[col].mean() + df[col].std()*3
    sigma3N = df[col].mean() - df[col].std()*3
    df[col] = df[col][df[col].between(sigma3N, sigma3P)]

In [14]:
def dropOutliersZscore(df, cols):
    for col in cols:
        dropOutlierZscore(df, col)

In [18]:
def stats_desc(df):
    print(df.agg(agg_func))

In [20]:
# by default apply IQR method, which seems to be faster and more efficient than Z-score
def removeOutlier(df, col, zscore = False):
    if (zscore):
        dropOutliersZscore(df, [col])
    else:
        dropOutliersIQR(df, [col])

In [21]:
def removeOutliers(df, cols, zscore = False):
    for col in cols:
        removeOutlier(df, col, zscore)

In [3]:
def extractCountry(s):
    s = str(s).upper()
    if (s.find(':') != -1):
        return s[str(s).find(':')+1:]
    else:
        return s
        
def transformCountryCode(s):
    try:
        return pycountry.countries.get(name=s).alpha_2
    except BaseException as err:
#         print(f"corresponding country code for {s} not found")
        return s[:2]

def cleanCategory(s):
    s1 = extractCountry(s)
    s2 = transformCountryCode(s1)
    return s2

# test
# print(cleanCategory('en:France'))
# print(cleanCategory('en'))
# print(cleanCategory('en:United States'))
# print(cleanCategory("france"))
# print(cleanCategory("United States"))
# print(cleanCategory("magyarország"))

In [13]:
# plots boxplots for each column
def plotBoxes(df, cols):
    size = cols.size
    plt.figure(figsize=(20,size*2.5))
    plt.title(f'analysis of {cols}')
    for i, col in enumerate(cols):
        plt.subplot(size, 1, i+1)
        df[col].plot(kind='box', vert=False)
    plt.show

In [22]:
# plots histogram for each columns
def plotDist(df, cols):
    size = cols.size
    plt.figure(figsize=(20,size*2.5))
    print('distribution of quantitative values')
    for i, col in enumerate(cols):
        # sns.displot(df, x=col, bins=200)
        plt.subplot(size, 1, i+1)
        df[col].plot(kind='hist', bins=200, title=f'{col}')
    plt.show